In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.5`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import $ivy.$
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}

In [3]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .config("spark.log.level", "WARN")
    .getOrCreate()
}

import spark.implicits._

Loading spark-stubs

Creating SparkSession

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/13 16:26:55 INFO SparkContext: Running Spark version 3.5.5
25/03/13 16:26:55 INFO SparkContext: OS info Mac OS X, 15.3.2, aarch64
25/03/13 16:26:55 INFO SparkContext: Java version 11.0.26
25/03/13 16:26:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@7333d64a
import spark.implicits._

In [4]:
spark.version

res4: String = "3.5.5"

## Create Dataset

### Из коллекции

In [5]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Person(name: String, age: Long)

val data = Seq(Person("Andy", 32L), Person("Mike", 25L), Person("Piter", 40L))

defined class Person
data: Seq[Person] = List(
  Person("Andy", 32L),
  Person("Mike", 25L),
  Person("Piter", 40L)
)

In [6]:
val ds = data.toDS()
ds.show

+-----+---+
| name|age|
+-----+---+
| Andy| 32|
| Mike| 25|
|Piter| 40|
+-----+---+



ds: Dataset[Person] = [name: string, age: bigint]

In [7]:
val ds2 = spark.createDataset(data)
ds2.show

+-----+---+
| name|age|
+-----+---+
| Andy| 32|
| Mike| 25|
|Piter| 40|
+-----+---+



ds2: Dataset[Person] = [name: string, age: bigint]

In [8]:
ds.schema

res8: types.StructType = StructType(
  StructField("name", StringType, true, {}),
  StructField("age", LongType, false, {})
)

In [9]:
ds.schema.equals(ds2.schema)

res9: Boolean = true

### Из внешнего источника

In [10]:
val peopleDF = spark.read.json("data/people.json")
val peopleDS = peopleDF.as[Person]

peopleDS.show()

json at cmd10.sc:1

show at cmd10.sc:4

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



peopleDF: DataFrame = [age: bigint, name: string]
peopleDS: Dataset[Person] = [age: bigint, name: string]

## Основные операции

In [11]:
val names = ds.map(_.name)
names.show

show at cmd11.sc:2

show at cmd11.sc:2

+-----+
|value|
+-----+
| Andy|
| Mike|
|Piter|
+-----+



names: Dataset[String] = [value: string]

In [12]:
val age30 = ds.filter(_.age > 30)
age30.show

show at cmd12.sc:2

show at cmd12.sc:2

+-----+---+
| name|age|
+-----+---+
| Andy| 32|
|Piter| 40|
+-----+---+



age30: Dataset[Person] = [name: string, age: bigint]

In [13]:
val age3 = ds.filter(_.age > 30).map(_.age)
age3.show

show at cmd13.sc:2

show at cmd13.sc:2

+-----+
|value|
+-----+
|   32|
|   40|
+-----+



age3: Dataset[Long] = [value: bigint]

In [14]:
val df1 = ds.select(col("name"), expr("age + 1").as("age1"))
df1.show

+-----+----+
| name|age1|
+-----+----+
| Andy|  33|
| Mike|  26|
|Piter|  41|
+-----+----+



df1: DataFrame = [name: string, age1: bigint]

In [15]:
val ds2 = ds.select(col("name"), expr("age + 1").as("age")).as[Person]
ds2.show

+-----+---+
| name|age|
+-----+---+
| Andy| 33|
| Mike| 26|
|Piter| 41|
+-----+---+



ds2: Dataset[Person] = [name: string, age: bigint]

In [16]:
val ds3 = ds.map(p => Person(p.name, p.age + 1))
ds3.show

show at cmd16.sc:2

show at cmd16.sc:2

+-----+---+
| name|age|
+-----+---+
| Andy| 33|
| Mike| 26|
|Piter| 41|
+-----+---+



ds3: Dataset[Person] = [name: string, age: bigint]

In [17]:
val jsonDS = ds.toJSON

jsonDS: Dataset[String] = [value: string]

In [18]:
jsonDS.show(false)

show at cmd18.sc:1

show at cmd18.sc:1

+-------------------------+
|value                    |
+-------------------------+
|{"name":"Andy","age":32} |
|{"name":"Mike","age":25} |
|{"name":"Piter","age":40}|
+-------------------------+

